In [41]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Carregar o arquivo CSV
atual_dir = os.getcwd()
parent_dir = os.path.split(atual_dir)
parent_dir = os.path.split(parent_dir[0])
parent_dir = os.path.split(parent_dir[0])
caminho_csv = os.path.join(parent_dir[0], "Pre-processamento\\noticias_dados_limpos.csv")
df = pd.read_csv(caminho_csv)

# Obter as categorias
categorias = df['Categoria'].unique()

# Carregar o modelo pré-treinado
model_path = "C:\\Users\\mathe\\OneDrive\\Área de Trabalho\\Detector\\Modelos\\BERT\\Treinamento"
model = BertForSequenceClassification.from_pretrained(model_path)

# Carregar o tokenizador
tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

# Texto para fazer a previsão
texto = """
'Não acho que a gente tenha problema no Congresso', diz Lula sobre articulação política
Questionado, presidente minimizou reclamações de parlamentares e evitou detalhar reunião fora da agenda com Arthur Lira. Lula também negou que tenha intenção de trocar ministros.
O presidente Luiz Inácio Lula da Silva (PT) negou nesta terça-feira (23) que o governo tenha problemas de articulação política no Congresso Nacional – apontados, nos últimos meses, pelos próprios parlamentares.
Lula afirmou, ainda, que está em uma "situação de muita tranquilidade na relação com o Congresso Nacional", e previu placares vitoriosos para as próximas votações.

O cenário indicado até o momento, no entanto, é de possíveis revezes em vetos – como no projeto que restringe as "saidinhas" de presos – e de pressão pela aprovação de pautas-bomba como a PEC do Quinquênio, que pode gerar custos adicionais de até R$ 82 bilhões nos próximos três anos.

"""

texto = texto.lower()

# Tokenizar o texto
inputs = tokenizer(texto, return_tensors="pt", truncation=True, max_length=128)

# Fazer a previsão
with torch.no_grad():
    outputs = model(**inputs)

# Obter as previsões
logits = outputs.logits
predicted_class_idx = logits.argmax().item()

# Mapear o índice da classe prevista para a categoria
# Cria um objeto LabelEncoder
le = LabelEncoder()

# Transforma os labels para variáveis categóricas
df['label'] = le.fit_transform(df['Categoria'])

original_class = le.inverse_transform([predicted_class_idx]) 
print(f"Classe original: {original_class[0]}")



Classe original: Real


In [42]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import os
import pandas as pd

# Carregar o arquivo CSV
atual_dir = os.getcwd()
parent_dir = os.path.split(atual_dir)
parent_dir = os.path.split(parent_dir[0])
parent_dir = os.path.split(parent_dir[0])
caminho_csv = os.path.join(parent_dir[0], "Pre-processamento\\noticias_dados_limpos.csv")
df = pd.read_csv(caminho_csv)

# Obter as categorias
categorias = df['Categoria'].unique()

# Carregar o modelo pré-treinado
model_path = "C:\\Users\\mathe\\OneDrive\\Área de Trabalho\\Detector\\Modelos\\BERT\\Treinamento"
model = BertForSequenceClassification.from_pretrained(model_path)

# Carregar o tokenizador
tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

# Texto para fazer a previsão
texto = """
Tubarões voadores são descobertos no oceano Atlântico. Um grupo de cientistas marinhos relatou a descoberta de uma nova espécie de tubarões voadores nas águas profundas do oceano Atlântico. Esses tubarões têm a habilidade única de se elevar acima da superfície do mar e planar por longas distâncias. A descoberta desafia as noções convencionais sobre a biologia marinha e promete revolucionar nossa compreensão dos ecossistemas oceânicos.
"""

texto = texto.lower()

# Tokenizar o texto
inputs = tokenizer(texto, return_tensors="pt", truncation=True, max_length=128)

# Fazer a previsão
with torch.no_grad():
    outputs = model(**inputs)

# Obter as previsões
logits = outputs.logits
probabilidades = torch.softmax(logits, dim=1)[0]

# Obter o índice da classe com a maior probabilidade
predicted_class_idx = torch.argmax(probabilidades).item()

# Cria um objeto LabelEncoder
le = LabelEncoder()

# Transforma os labels para variáveis categóricas
df['label'] = le.fit_transform(df['Categoria'])

original_class = le.inverse_transform([predicted_class_idx]) 
print(f"Classe original: {original_class[0]}")



Classe original: Falso
